# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Behavioral Cloning

### Step 0: Declare and import dependencies on modules. Also declare global variables

In [1]:
import csv
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
import cv2
import numpy as np

Using TensorFlow backend.


In [2]:
data_path = "recorded-data"
generator_batch_size = 5
steering_offset = 0.25

### Step 1: Load The Data

#### 1. Load lines from csv

In [3]:
def load_lines(path):
    lines = []
    with open(path + "/driving_log.csv") as datafile:
        reader = csv.reader(datafile)
        for line in reader:
            lines.append(line)
    return lines

#### 2. Extract center, left and right images

In [4]:
def extract_images(line):
    center_image = cv2.imread(data_path + "/IMG/" + line[0].split("\\")[-1])
    left_image = cv2.imread(data_path + "/IMG/" + line[1].split("\\")[-1])
    right_image = cv2.imread(data_path + "/IMG/" + line[2].split("\\")[-1])
    return (center_image, left_image, right_image)

#### 3. Extract steering angles

In [5]:
def extract_steering_angles(line):
    steering_angle_center = float(line[3])
    steering_angle_left = steering_angle_center + steering_offset
    steering_angle_right = steering_angle_center - steering_offset
    return (steering_angle_center, steering_angle_left, steering_angle_right)

#### 4. Load data without generator

In [6]:
def get_data_without_generator(path, lines):
    images = []
    steering_angles = []
    for line in lines:
        center_image, left_image, right_image = extract_images(line)
        steering_angle_center, steering_angle_left, steering_angle_right = extract_steering_angles(line)
        images.extend([center_image, left_image, right_image])
        steering_angles.extend([steering_angle_center, steering_angle_left, steering_angle_right])
    return np.array(images), np.array(steering_angles)

#### 5. Load data with generator

In [15]:
def get_data_generator(lines, generator_batch_size=100):
    offset = 0
    shuffled_lines = shuffle(lines)
    while 1:
        images = []
        steering_angles = []
        line_batch = shuffled_lines[offset : offset + generator_batch_size]
        for line in line_batch:
            center_image, left_image, right_image = extract_images(line)
            steering_angle_center, steering_angle_left, steering_angle_right = extract_steering_angles(line)
            images.extend([center_image, left_image, right_image])
            steering_angles.extend([steering_angle_center, steering_angle_left, steering_angle_right])
        offset += generator_batch_size
        if (offset >= len(lines)):
            offset = 0
            shuffled_lines = shuffle(lines)
        yield (np.array(images), np.array(steering_angles))

### Step 2: Train model

#### 1. Training using custom network architecture

In [8]:
def train_model_custom(training_generator, validation_generator, batch_size=50, epochs=10, use_generator=True):
    model = Sequential()
    model.add(Flatten(input_shape=(160, 320, 3)))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    #model.fit(training_generator, validation_generator, validation_split=0.3, shuffle=True, nb_epoch=20)
    model.fit_generator(training_generator, nb_epoch=epochs, samples_per_epoch=batch_size)
    model.save("CarND-Behavioral-Cloning-P3/model-custom-network.h5")

#### 1. Training using LeNet

In [27]:
def train_model_lenet(training_generator, validation_generator, nb_training, nb_validation, epochs=10, use_generator=True):
    model = Sequential()
    model.add(Convolution2D(6, 5, 5, border_mode='valid', input_shape=(160, 320, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D((2, 2)))
    model.add(Convolution2D(16, 5, 5, border_mode='valid', input_shape=(156, 316, 6)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten(input_shape=(152, 312, 16)))
    model.add(Dense(120))
    model.add(Activation("relu"))
    model.add(Dense(84))
    model.add(Activation("relu"))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    if use_generator:
        model.fit_generator(training_generator, validation_data=validation_generator, nb_epoch=epochs, samples_per_epoch=(nb_training+nb_validation), nb_val_samples=nb_validation)
    else:
        model.fit(training_generator[0], training_generator[1], validation_split=0.2, shuffle=True, nb_epoch=epochs)
    model.save("CarND-Behavioral-Cloning-P3/model-lenet.h5")

### Step 3: Run the model

In [29]:
def main():
    lines = load_lines(data_path) 
    training_set_lines, validation_set_lines = train_test_split(lines, test_size=0.2)
    #training_set_lines = lines[0:50]
    #validation_set_lines = lines[50:60]
    nb_training = len(training_set_lines)
    nb_validation = len(validation_set_lines)

    training_generator = get_data_generator(training_set_lines, generator_batch_size=generator_batch_size)
    validation_generator = get_data_generator(validation_set_lines, generator_batch_size=generator_batch_size)
    
    train_model_lenet(training_generator, validation_generator, nb_training, nb_validation, epochs=4, use_generator=True)
    #train_model(training_generator, validation_generator, batch_size=batch_size, epochs=4)

    #training_images, steering_angles = get_data_without_generator(data_path, lines)
    #train_model_lenet((training_images, steering_angles), None, epochs=4, use_generator=False)
main()

Epoch 1/4
 300/1736 [====>.........................] - ETA: 113s - loss: 169005.1271 

KeyboardInterrupt: 